This Notebook calculate the scattering & intrinsic absorption parameters of the Rayleigh waves following the instruction proposed by Hirose et al. (2019). **This notebook demonstrates single-station measurements**

**Publication related to this script:** 
Hirose, T., Nakahara, H., & Nishimura, T. (2019). A passive estimation method of scattering and intrinsic absorption parameters from envelopes of seismic ambient noise cross‐correlation functions. Geophysical Research Letters, 46(7), 3634-3642. https://doi.org/10.1029/2018GL080553


Step: <br>
1) Data preparation<br>
2) Calculation of mean-squared (MS) envelopes --> observed energy densities (Eobs)<br>
3) Read in the windows using for measuring attenuation
4) Calculation of synthesized energy densities (Esyn) via a grid search<br>
5) Determination of best-fit parameters: intrinsic absorption parameter *b* (for single station)<br>
6) Plotting results

In [ ]:
import os
import glob
import numpy as np

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import h5py
import pandas as pd

from Esyn_func import *

#### Step 1 ---  Data preparation  

In [ ]:
# detremine the h5 file for processing
path="/home/kffeng/DATA_wsd/FarmDAS/"   # Insert the data path
dataname="autocorr_25_50Hz.hdf5" # Insert data file name, i.e. 'autocorr_25_50Hz.hdf5'
sfiles = sorted(glob.glob(os.path.join(path, dataname)))
print("The h5 file: ",sfiles)
fname=sfiles[0].split("FarmDAS/")[1].split(".hdf5")[0]
print("Processed station pair: ",fname)

In [ ]:
# setting up the configuration parameter for the prepared h5 file for processing
comp_arr=['autocorr']
num_cmp=len(comp_arr)
fnum=1      # file number
nchn=51     # channel number
# information of the input data:  lagtime (lag), sampling rate (samp)
lag=5 
samp=500
leng=int(lag*samp*2+1)
npts=leng
print("Lag-time: ",lag,", sampling rate: ",samp,", total data length in points: ",leng)

In [ ]:
# Ready to read in h5 file as stackf
stackf=np.zeros((nchn, fnum,num_cmp,2,leng))  
print(stackf.shape)
vdist=np.zeros((fnum,1))  # S-R distance array


In [ ]:
# Read in h5 file as autocorr
h5f=h5py.File(sfiles[0],'r')
autocorr=h5f['autocorr']
nlocs, nstartf, n_lag, n_pair = autocorr.shape

locs=np.arange(44,44+nlocs)
startf=np.arange(nstartf)

In [ ]:
tlag=500
short_ncf=np.zeros((nlocs,nstartf,tlag+1), dtype = np.float32)
print(short_ncf.shape, autocorr.shape)
for chan in range(nlocs):
    chnm=locs[chan]
    for tsp in range(nstartf):
        short_ncf[chan,tsp, : ] =autocorr[chan, tsp, ((n_lag-1)//2):((n_lag-1)//2)+tlag+1].reshape(tlag+1)

    # #print(np.min(short_ncf[chan]),np.max(short_ncf[chan]))
    # plt.figure(figsize=(6,3))
    # plt.imshow(short_ncf[chan], extent=[0,tlag/samp,nstartf,0],aspect='auto',vmin=-10**3.5,vmax=10**3.5,cmap = 'RdBu')
    # plt.title("Channel "+str(chnm))
    # plt.ylabel('Window number')
    # plt.xlabel('Lag-time (sec)')
    # plt.tight_layout()


#### Step 2 --- Calculation of mean-squared (MS) envelopes --> ***Eobs*** 

In [ ]:
# smooth 
nccomp=1
freq1=25
freq2=50
winlen=0.05  # smoothing window length
dt=1/samp

nwin=nstartf 
num_chunk=nstartf
num_segmts=1
npts_segmt=tlag+1


msv=np.zeros((nlocs, num_chunk*num_segmts, npts_segmt))
ptime=np.arange(0,(tlag+1)/samp,dt)
# get all components average
for chan in range(nlocs):
    chnm=locs[chan]
    for tsp in range(nstartf):
        para = { 'winlen':winlen, 'dt':dt , 'npts':tlag+1}
        msv[chan,tsp]=get_smooth(short_ncf[chan,tsp], para)
        msv[chan,tsp]=msv[chan,tsp]/np.max(msv[chan,tsp])

    fig,ax=plt.subplots(1,4,figsize=(22,3))
    ax[0].imshow(short_ncf[chan], extent=[0,tlag/samp,nstartf,0],aspect='auto',vmin=-10**(3.5),vmax=10**(3.5),cmap = 'RdBu')
    ax[0].set_title("Channel "+str(chnm))
    ax[0].set_ylabel('Window number')
    ax[0].set_xlabel('Lag-time (sec)')
    
    ax[1].imshow(msv[chan], extent=[0,tlag/samp,nstartf,0],aspect='auto',vmin=0,vmax=1,cmap = 'RdBu')
    ax[1].set_title("Channel "+str(chnm))
    ax[1].set_ylabel('Window number')
    ax[1].set_xlabel('Lag-time (sec)')
    
    ax[2].set_xlim(ptime[0],ptime[-1])
    for k in range(nstartf):
        ax[2].plot( ptime,msv[chan,k,:])
    ax[2].set_title("Channel "+str(chnm)+", Mean-squared amplitude")
    ax[2].set_ylabel('Amplitude')
    ax[2].set_xlabel('Lag-time (sec)') 
    
    ax[3].set_yscale('log')
    ax[3].set_ylim(1e-3,2)
    ax[3].set_xlim(ptime[0],ptime[-1])
    for k in range(nstartf):
        ax[3].plot( ptime,msv[chan,k,:])
    ax[3].set_title("Channel "+str(chnm)+", Mean-squared amplitude")
    ax[3].set_ylabel('Log10(amplitude)')
    ax[3].set_xlabel('Lag-time (sec)')
    


#### Step 3 ---  Read in the windows using for measuring attenuation

In [ ]:
# Read in the attenuation measuring window
codafn='Attenuation_Window_over_Chan.csv'
coda_df=pd.read_csv(codafn)

twinbe=np.zeros((nchn,2))
for chan in range(nlocs):
    twinbe[chan,0]=coda_df['tbeg'][chan]
    twinbe[chan,1]=coda_df['tend'][chan]


#### Step 4 ---  Calculation of synthesized energy densities (*Esyn*) via a grid search

In [ ]:
vdist=np.zeros((1))     # station distance
cvel=2.6                # Rayleigh wave velocities over the freqency bands
mfpx=np.zeros(1)        # mean_free_path search array
intby=np.zeros(500)      # intrinsic_b search array


In [ ]:
# getting the sum of squared residuals (SSR) between Eobs and Esyn  
SSR_final=np.zeros((len(mfpx),len(intby)))
SSR=np.zeros((nlocs,nwin,1,len(mfpx),len(intby)))
Qiall=np.zeros((nlocs,nwin,len(intby)))

fmax=freq2
fmin=freq1
c=cvel
vdist=[0.000001]  # To avoid zero value at denominator
for chan in range(nlocs):    
    twin_in=twinbe[chan]
    for ntw in range(nwin):
        data=np.zeros(shape=(1,2,tlag+1))
        data[0,0,:]=ptime
        data[0,1,:]=msv[chan,ntw,:]
        #print(data.shape,fmsv_mean[ntw].shape)
        # parameters for getting the sum of squared residuals (SSR) between Eobs and Esyn 
        para={ 'fb':0 , 'vdist':vdist, 'npts': tlag+1, 'dt':dt, 'cvel':c, \
            'mfp':mfpx, 'intb':intby,'twin':twin_in, 'fmsv':data }
        # call function get_SSR
        SSR_final, mfpx, intby = get_SSR(1, para )
        del data
        SSR[chan,ntw,0,:,:]=SSR_final
        
        loc = np.where(SSR_final.T == np.amin(SSR_final.T))
        #plt.plot(intby,SSR_final[0,:],'.',label=locs[chan])
        #plt.xlabel("intrinsic b")
        #plt.ylabel("Normalized SSR")
        #plt.grid(True)
        
        Qiall[chan,ntw]=(2.0*np.pi*((fmin+fmax)/2.0))/intby
         
    #plt.legend()
print(SSR.shape)

In [ ]:
def plot_fitting_result(mean_free,intrinsic_b,tt,Eobs,Esyn,fname,dist,twind,fmin,fmax,win_num):
    plt.figure(figsize=(4,2))
    plt.yscale('log', base=10)
    
    pymax=np.max(Eobs)*5
    pymin=pymax/1e6
    print(" Channel: %s,", fname, pymin , pymax )
    plt.ylim( pymin , pymax )
    plt.plot( tt, Eobs, "k-", linewidth=1)
    plt.plot( tt, Esyn, "b--", linewidth=1)
    plt.plot([twind[0],twind[0],twind[-1],twind[-1],twind[0]],[pymin, pymax,pymax,pymin,pymin],"r", linewidth=2)
    Qi=(2.0*np.pi*((fmin+fmax)/2.0))/intrinsic_b

    plt.title("channel %s @%4.2f-%4.2f Hz, \nintrinsic b: %.2f, Qi: %.2f, Window no. %d" \
            % ( fname,fmin,fmax,intrinsic_b, Qi, win_num))
    plt.xlabel("Lag time (sec)")
    plt.ylabel("Energy density Amp")
    plt.tight_layout()   
    plt.show()
                    

#### Step 5 --- Determination of best-fit parameters

In [ ]:
# getting the optimal value from the SSR
result_intb=np.zeros((nlocs,nwin,1))
result_mfp=np.zeros((nlocs,nwin, 1))
result_intQ=np.zeros((nlocs,nwin,1))


Eobs=np.ndarray((1,tlag+1))
Esyn=np.ndarray((1,tlag+1))
aa=0
r=np.take(vdist[aa],0) 

fmin=freq1
fmax=freq2
wfcen=2.0*np.pi*((freq1+freq2)/2.0)
for chan in range(nlocs):
    chnm=locs[chan]
    twin_in=twinbe[chan]
    for ntw in range(nwin):

        data=np.zeros(shape=(1,2,tlag+1))
        data[0,0,:]=ptime
        data[0,1,:]=msv[chan,ntw,:]
        # parameters for getting optimal value from the sum of squared residuals (SSR) between Eobs and Esyn 
        para={ 'fb':0, 'fmin':fmin, 'fmax':fmax, 'vdist':vdist, 'npts':tlag+1, 'dt':dt, 'cvel':c, 'filenum':aa, \
            'mfp':mfpx, 'intb':intby,'twin':twin_in, 'fmsv':data, 'SSR':SSR[chan,ntw] , 'sta':chnm}
        # call function get_optimal
        result_intb[chan,ntw], result_mfp[chan,ntw], Eobs, Esyn = get_optimal(1,para)
        # plotting fitting results

        # if ntw == 50:
        #     plot_fitting_result(result_mfp[chan,ntw],result_intb[chan,ntw],ptime[:-25], 
        #                 Eobs[:-25],Esyn[:-25],chnm,vdist[0],twin_in,fmin,fmax,ntw)
        # result_intQ[chan,ntw]=(2.0*np.pi*((fmin+fmax)/2.0))/result_intb[chan,ntw]




#### Step 6 --- Plotting results

In [ ]:
SSRgrid=np.zeros((nlocs,nwin,len(intby)))


from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
from matplotlib.colors import LogNorm

vmin=1
vmax=1.1

aspect = 20
pad_fraction = 0.8


for chan in range(nlocs):
    SSRgrid[chan]=SSR[chan,:,0,0,:]
    
    fig=plt.figure(figsize=(8,2))
    ax=plt.gca()
    plt.sca(ax)
    im=ax.imshow(SSRgrid[chan].T, cmap='hot_r', vmin=1, vmax=1.1, aspect='auto')
    ax.invert_yaxis()
    ax.set_ylim(0,960)
    ax.set_ylabel("intrinsic b (x10)")
    ax.set_xlabel("window number")
    ax.set_title("Channel "+str(locs[chan])+" : Grid-searching on intrinsic b (by SSR)")
    ax.grid(True, color='white', linestyle='--')
    
    divider = make_axes_locatable(ax)
    width = axes_size.AxesY(ax, aspect=1./aspect)
    pad = axes_size.Fraction(pad_fraction, width)
    cax = divider.append_axes("right", size=width, pad=pad)
    plt.colorbar(im, cax=cax, label="SSR/min(SSR)" , spacing='proportional')
    plt.tight_layout()

In [ ]:
for chan in range(nlocs):
    chnm=locs[chan]
    fig,ax=plt.subplots(2,1,figsize=(6,4))
    ax[0].plot(result_intb[chan], label='intrinsic b')
    ax[0].legend();ax[0].grid(True)
    ax[0].set_title('Channel '+str(chnm)+' Estimated intrinsic parameter b')
    ax[0].set_xlabel('Window number')
    ax[0].set_ylabel('intrinsic b')

    ax[1].plot(result_intQ[chan], label='Qi')
    ax[1].legend();ax[1].grid(True)
    ax[1].set_ylim(0,120)
    ax[1].set_title('Channel '+str(chnm)+' Estimated intrinsic Q')
    ax[1].set_xlabel('Window number')
    ax[1].set_ylabel('Qi value')
    plt.tight_layout()